In [1]:
import json
from enum import Enum
from collections import namedtuple
from ipynb.fs.full.Critical_Section import *
import numpy as np

In [2]:
class lock_event:
    
    def __init__(self,n_thr):
        self.lock_clock = np.zeros(n_thr)
        self.objects = {}
    
    

### Read the events from log

In [5]:
events, critical_section_list,num_threads,lock_map = gen_event_cs2("temp.log")
for x in critical_section_list:
    print(x)
    
print(num_threads,lock_map)

Thread : 1 
Locks : set() 
Read Objects : {'phi_x'} 
Write Objects : set() 
Events : {0} 
Child Sections : set()

Thread : 1 
Locks : set() 
Read Objects : {'phi_y'} 
Write Objects : set() 
Events : {1} 
Child Sections : set()

Thread : 1 
Locks : {'l', 'm', 'n'} 
Read Objects : {'xl', 'x', 'a'} 
Write Objects : {'yl', 'b', 'y'} 
Events : {2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17} 
Child Sections : {3, 4, 5}

Thread : 1 
Locks : {'l', 'n'} 
Read Objects : {'xl', 'x'} 
Write Objects : {'yl', 'y'} 
Events : {5, 6, 7, 8, 9, 10, 11, 12} 
Child Sections : {4}

Thread : 1 
Locks : {'n'} 
Read Objects : {'xl'} 
Write Objects : {'yl'} 
Events : {8, 9, 10, 11} 
Child Sections : set()

Thread : 1 
Locks : {'n'} 
Read Objects : {'xl'} 
Write Objects : {'yl'} 
Events : {16, 13, 14, 15} 
Child Sections : set()



In [6]:
thread_cp = {}